# `01_preprocess.ipynb`

## 0.1 - Setup

### 0.1.1 - Package Imports

In [29]:
# imports from python standard library
import re
from pathlib import Path

# data science packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# huggingface packages

### 0.1.2- Constants

In [2]:
DATA_DIR = Path("./data")
DATA_DIR_RAW = DATA_DIR / "raw"
TRANSCRIPT_DATA = DATA_DIR_RAW / "transcript/The-Office-Lines-V4.csv"

### 0.1.3 - Options

In [3]:
# set plotting backend for Pandas
pd.set_option('plotting.backend', 'plotly')

## 0.2 Import Data

In [4]:
script_df = pd.read_csv(
    filepath_or_buffer=TRANSCRIPT_DATA,
    header=0,
    usecols=['season', 'episode', 'title', 'scene', 'speaker', 'line']
    )

script_df.sample(20)

,season,episode,title,scene,speaker,line
32512,6,23,Body Language,4953,Michael,"Oh, thanks. You're not bad, either."
22519,5,16,Lecture Circuit (Part 1),3621,Dwight,That's how much you and I contributed! I... D...
27384,6,4,Niagara (Parts 1&2),4249,Isabel,Um... nothing. Anymore.
34743,7,5,The Sting,5280,Dwight,What are you doing?
37487,7,14,The Seminar,5645,Holly,"Oh, I don't know, Michael. I'm not feeling up ..."
52491,9,18,Promos,7802,Jim,Let's go to the conference room.
14458,4,3,Dunder Mifflin Infinity (Parts 1&2),2454,Michael,We don't need luck.
17100,4,15,Night Out,2867,Michael,Smells good.
37748,7,15,The Search,5683,Jim,Where'd I put what?
26991,6,4,Niagara (Parts 1&2),4199,Michael,Those glasses are super dark.


# 1 - EDA

## 1.1 - Speaker Frequency

In [5]:
# no filtering for seasons
print("frequency:\n\n", script_df['speaker'].value_counts(normalize=False).nlargest(10), sep="")
print("\ntotal of frequency (top 10):\t", script_df['speaker'].value_counts(normalize=False).nlargest(10).sum(), sep="")
print("total of frequency (all):  \t", script_df['speaker'].value_counts(normalize=False).sum(), sep="")
print("\n", "-"*20)
print("\nnormalized:\n\n", script_df['speaker'].value_counts(normalize=True).nlargest(10), sep="")
print("\ntotal of normalized:\t", script_df['speaker'].value_counts(normalize=True).nlargest(10).sum(), sep="")

frequency:

Michael    10773
Dwight      6752
Jim         6222
Pam         4973
Andy        3698
Kevin       1535
Angela      1534
Erin        1413
Oscar       1336
Ryan        1182
Name: speaker, dtype: int64

total of frequency (top 10):	39418
total of frequency (all):  	54626

 --------------------

normalized:

Michael    0.197214
Dwight     0.123604
Jim        0.113902
Pam        0.091037
Andy       0.067697
Kevin      0.028100
Angela     0.028082
Erin       0.025867
Oscar      0.024457
Ryan       0.021638
Name: speaker, dtype: float64

total of normalized:	0.7215977739537949


The top 10 speakers account for ~72% of all lines in the show, with Michael having the highest proportion of individual utterances (just shy of 20%).

In [6]:
top_10_speaker_list = script_df['speaker'].value_counts(normalize=True).nlargest(10).index.tolist()
top_10_speaker_df = script_df.loc[script_df['speaker'].isin(top_10_speaker_list)]

top_10_speaker_df = top_10_speaker_df.astype({'season': str})

top_10_groupedby_season = top_10_speaker_df.groupby(by=['season'])['speaker'].value_counts(normalize=True).unstack('speaker')
top_10_groupedby_speaker = top_10_speaker_df.groupby(by=['speaker'])['season'].value_counts(normalize=True).unstack('season')

In [7]:
title = "Distribution of Each Seasons' Speakers"
hlabel = "Frequency of utterances (normalized)"
vlabel = "Season #"
legend_label = "Speaker"

template = 'plotly_white'
colormap = px.colors.qualitative.D3

width = 1000; height = 500; margin = 110

fig = top_10_groupedby_season.plot.barh(color_discrete_sequence=colormap)

fig.update_layout(
    template=template,
    title=dict(text=title, font=dict(size=20)),
    xaxis_title=dict(text=hlabel, font=dict(size=16)),
    yaxis_title=dict(text=vlabel, font=dict(size=16)),
    legend_title=dict(text=legend_label, font=dict(size=16)),
    margin=dict(l=margin, r=margin, t=margin, b=margin),
    width=width, 
    height=height,
    font=dict(family='Open Sans, Arial', color='black')
)

fig.show()

That's one view of per-season speakers, albeit somewhat challenging to interpret given there are a different number of total utterances per season.

What about each character's speaking per season:

In [8]:
title = "Distribution of Each Speaker's Lines Across Seasons"
hlabel = "Frequency of utterances (normalized)"
vlabel = "Speaker (Limited to Top 10 Speakers)"
legend_label = "Season #"

template = 'plotly_white'
colormap = px.colors.qualitative.D3

width = 1000; height = 500; margin = 110

fig = top_10_groupedby_speaker.plot.barh(color_discrete_sequence=colormap)

fig.update_layout(
    template=template,
    title=dict(text=title, font=dict(size=20)),
    xaxis_title=dict(text=hlabel, font=dict(size=16)),
    yaxis_title=dict(text=vlabel, font=dict(size=16)),
    legend_title=dict(text=legend_label, font=dict(size=16)),
    margin=dict(l=margin, r=margin, t=margin, b=margin),
    width=width, 
    height=height,
    font=dict(family='Open Sans, Arial', color='black')
)

fig.show()

# 2 - Preprocess

## 2.1 - Data Cleaning (Pipeline Setup)

Using a pipeline approach for this, with `sklearn.pipeline.Pipeline` and `sklearn.preprocessing.FunctionTransformer` organizing the effort.

In [133]:
data_cleaning_pipeline = Pipeline(steps=[], verbose=True)

### 2.1.1 - `speaker` - Lowercase values

Set all values of `speaker` column to lowercase.

In [134]:
def lowercase_speaker(df: pd.DataFrame) -> pd.DataFrame:
    # copy df to not modify original
    new_df = df.copy()

    new_df['speaker'] = new_df['speaker'].apply(lambda s: s.lower())

    return new_df

In [123]:
# sanity check for lowercase_speaker()
print("[before]\n", script_df['speaker'].value_counts().nlargest(10).index.tolist())

print("[after]\n",lowercase_speaker(script_df)['speaker'].value_counts().nlargest(10).index.tolist())

[before]
 ['Michael', 'Dwight', 'Jim', 'Pam', 'Andy', 'Kevin', 'Angela', 'Erin', 'Oscar', 'Ryan']
[after]
 ['michael', 'dwight', 'jim', 'pam', 'andy', 'kevin', 'angela', 'erin', 'oscar', 'ryan']


In [135]:
data_cleaning_pipeline.steps.append(
    ("lowercase_speaker", 
     FunctionTransformer(lowercase_speaker, kw_args=dict()))
)

### 2.1.2 - `speaker` - Find orphaned lines

Extra attention is given to the `speaker` column as this is going to be the basis of our class label.

Some values for `speaker` contain near-miss values for common speaker names, e.g. `Jim:` for `Jim`.

Test out some regex patterns for dialogue by `Jim` to understand extent:

In [100]:
test_speaker = 'jim'

script_df.loc[
    (script_df['speaker'].str.lower() != test_speaker)
    &
    (script_df['speaker'].str.lower().str.contains(test_speaker))
    , 'speaker'].value_counts()

Jim:                            33
Fake Jim                         6
Jim and Pam                      4
Andy and Jim                     4
Pam and Jim                      2
JIM9334                          2
Michael and Jim                  2
Jim                              2
Jim, Pam, Michael and Dwight     2
Jim & Dwight                     2
Jim, Josh, and Dwight            1
Pam/Jim                          1
Jim & Pam                        1
Dwight, Jim and Michael          1
 Jim                             1
Jim and Dwight                   1
Jim's Dad                        1
"Jim"                            1
Dwight, Erin, Jim & Kevin        1
Jim's voicemail                  1
Name: speaker, dtype: int64

Most common occurrence is "`Jim:`" but also have occurrences of group dialogue (containing commas, "and", "&"), possessive uses ("Jim's voicemail"), and one episode where Jim was speaking with Pam over an instant messaging program as "JIM9334" (highlighted in next cell, below).

In [10]:
script_df.loc[
    script_df['speaker'] == 'JIM9334'
]

,season,episode,title,scene,speaker,line
18756,5,1,Weight Loss (Parts 1&2),3145,JIM9334,Yeah. C'mon we'll meet halfway. You'll be back...
18761,5,1,Weight Loss (Parts 1&2),3145,JIM9334,The rest stop where that soda exploded on me. ...


Try to apply a regex search to recover some of the clearer examples to potentially associate them with "Jim".

Regex pattern chosen will:
 - be applied case-insensitive
 - allow preceding whitespace but otherwise expect "Jim" at start of string
 - disallow "Jim" to be followed by " and", " &", apostrophes, commas, or digits

In [74]:
test_speaker = 'jim'
test_pattern = r"^\s?" + test_speaker + r"(?!\s?and|'|,|\d|\s?&)"

script_df.loc[
    # first condition is to avoid simple/direct matches of "Jim"
    (script_df['speaker'].str.lower() != test_speaker)
    &
    (script_df['speaker'].str.contains(test_pattern, case=False))
    ]

,season,episode,title,scene,speaker,line
21870,5,13,Prince Family Paper,3524,Jim,This close.
24997,5,25,Broke,3955,Jim,You're just out of business?
31120,6,17,The Delivery (Parts 1&2),4755,Jim,Okay. Sshh. I know.
31800,6,21,Happy Hour,4846,Jim:,made a sale.
31809,6,21,Happy Hour,4846,Jim:,. I had a really hard work out this morning.
31823,6,21,Happy Hour,4849,Jim:,w.
31848,6,21,Happy Hour,4855,Jim:,"would love, love, love to go. Only problem is,..."
31850,6,21,Happy Hour,4855,Jim:,od one.
31852,6,21,Happy Hour,4855,Jim:,"solutely, I will do that right now."
31853,6,21,Happy Hour,4856,Jim:,"gotta tell you, this baby is amazing. She gets..."


It appears the "Happy Hour" episode from Season 6 may contain some errors in the dialogue. We may not want to include "Jim:" after all, to avoid adding noise to our data.

Try the same regex approach using Dwight to see how transferrable the approach is:

In [91]:
test_speaker = 'dwight'
test_pattern = r"^\s?" + test_speaker + r"(?!\s?and|'|,|\d|\s?&|:)"

script_df.loc[
    # first condition is to avoid simple/direct matches of test speaker
    (script_df['speaker'].str.lower() != test_speaker)
    &
    (script_df['speaker'].str.contains(test_pattern, case=False))
    ]

,season,episode,title,scene,speaker,line
8751,3,7,Branch Closing,1498,Dwight,"When you become close with someone, you develo..."
14746,4,5,Launch Party (Parts 1&2),2493,DwightKSchrute,You tell me.
14749,4,5,Launch Party (Parts 1&2),2493,DwightKSchrute,How do I know this isn't Jim?
14809,4,5,Launch Party (Parts 1&2),2499,DwightKSchrute,Here's a suggestion computer. I assume you re...
20103,5,7,Customer Survey,3321,Dwight,Customer service might be monitoring this conv...
32185,6,22,Secretary's Day,4904,Dwight.,"Hello, Pam."


We again see an instant messager screenname ("DwightKSchrute") appear, more problematic dialogue from S6E21 "Happy Hour", and otherwise ~3 lines that could be "saved".

For a Dwight/Not Dwight classifier, these three lines would represent $\frac{3}{(6,752+3)}$ or 0.044\% of Dwight's individual utterances and $\frac{3}{39,418+3}$ or 0.008\% of the top 10 character's utterances.

In [13]:
print(f"percent of Dwight's utterances: {3 / (6752+3) * 100:0.3f} %")
print(f"percent of top 10's utterances: {3 / (39418+3) * 100:0.3f} %")

percent of Dwight's utterances: 0.044 %
percent of top 10's utterances: 0.008 %


For this cleaning step, we'll add the colon to the exclusionary regex. Later, we'll examine S06E21 to see if its dialogue can be used at all.

In [82]:
def connect_orphaned_lines(df: pd.DataFrame, speaker_list: list[str]) -> pd.DataFrame:
    # copy incoming df to not modify original
    new_df = df.copy()
    
    for speaker in speaker_list:
        match_pattern = r"^\s?" + speaker + r"(?!\s?and|'|,|\d|\s?&|:)"

        new_df.loc[
            # first condition is to avoid simple/direct matches of test speaker
            (new_df['speaker'].str.lower() != speaker.lower())
            &
            # second condition matches the regex pattern 
            (new_df['speaker'].str.contains(match_pattern, case=False)),
            'speaker'] = speaker
    
    return new_df

In [143]:
# sanity check of connect_orphaned_lines()
test_speaker = "dwight"
test_pattern = r"^\s?" + test_speaker + r"(?!\s?and|'|,|\d|\s?&|:)"

print("[before] utterances by 'dwight' or 'Dwight':", 
      script_df.loc[(script_df.speaker.str.lower() == 'dwight'), 'speaker'].count())

sanity_df = connect_orphaned_lines(script_df, [test_speaker])

print("[after]  utterances by 'dwight' or 'Dwight':",
      sanity_df.loc[(sanity_df.speaker.str.lower() == 'dwight'), 'speaker'].count())

[before] utterances by 'dwight' or 'Dwight': 6752
[after]  utterances by 'dwight' or 'Dwight': 6758


In [136]:
top_10_speaker_list = script_df['speaker'].value_counts(normalize=True).nlargest(10).index.tolist()

data_cleaning_pipeline.steps.append(
    ("connect_orphaned_lines_top10", 
     FunctionTransformer(connect_orphaned_lines, kw_args=dict(speaker_list=top_10_speaker_list)))
)

### 2.1.3 - `season` / `episode` - Problematic Episodes

#### 2.1.3.1 - S06E21 "Happy Hour"

The text of this episode appears to have an issue in the dataset CSV file.

Here's an excerpt from the raw CSV file text showing a few lines of "good" data followed by the first few lines of this episode's data:

```
season,episode,title,scene,speaker,line,
...
6,20,New Leads,4844,Dwight,We have an awesome bean bag chair that's perfect for the break room. ,
6,20,New Leads,4844,Phyllis,Yuck. I'm not going to sit on that disgusting seat.,
6,20,New Leads,4844,Dwight,"Yeah, damn right you're not. 'Cause it's for me and Michael only ",
6,20,New Leads,4845,Andy, It's freezing out here.,
6,20,New Leads,4845,Erin, Go on. I have warm blood. ,
6,20,New Leads,4845,Andy,"Oh wow, thank you. You're the nicest person I've ever met. ",
6,21,Happy Hour,4846,Dwight: ,ichael grunts and strains while doing push-ups] Breathe. Work your core. Come on.,
6,21,Happy Hour,4846,Michael: ,w many is that?,
6,21,Happy Hour,4846,Dwight: ,"t counting the last one, 25.",
6,21,Happy Hour,4846,Michael: ,unt the last one.,
6,21,Happy Hour,4846,Dwight: ,", 25 and one girl push-up!",
6,21,Happy Hour,4846,Michael: ,", new record!",
...
```

The remainder of the dialogue from the "Happy Hour" episode shows the same types of aberrations:  
 - Speaker name ends with a colon "`:`" character followed by a space
 - The first character(s) of text from the line appear to be cut off
 - Some descriptive text, describing a non-spoken detail surrounding the line and enclosed in square brackets (`[]`), remains in the lines.
    - For example: "`ichael grunts and strains while doing push-ups]`"

The data for these lines are still distinguishable based on the numeric fields for `season` and `episode`, so they could be easily and accurately filtered out of our dataset.

In [137]:
def remove_episode(df: pd.DataFrame, season: int, episode: int) -> pd.DataFrame:
    """Removes the episode's rows from the overall dataframe."""
    mask_index = df.loc[
        (
            (df['season'] == season) 
            & 
            (df['episode'] == episode)
        )
    ].index

    return df.drop(index=mask_index)

In [28]:
# sanity check for remove_episode()
print("[before] episodes in season 6:\n", script_df.loc[script_df['season'] == 6, 'episode'].unique())

sanity_df = remove_episode(script_df, season=6, episode=21)

print("\n[after] episodes in season 6:\n", sanity_df.loc[script_df['season'] == 6, 'episode'].unique())

[before] episodes in season 6:
 [ 1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 19 20 21 22 23 24 25 26]

[after] episodes in season 6:
 [ 1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 19 20 22 23 24 25 26]


For now we're only removing this one episode, so adding this to the data cleaning pipeline.

In [138]:
data_cleaning_pipeline.steps.append(
    ("remove_episode_s6e21", 
     FunctionTransformer(remove_episode, kw_args=dict(season=6, episode=21)))
)

## 2.2 - Data Cleaning (Pipeline Execution)

In [139]:
# note: no 'fit' functionality, but with fit_transform and verbose=True this
#   prints some nice output of the pipeline steps
script_df_cleaned = data_cleaning_pipeline.fit_transform(script_df)

script_df_cleaned.head()

[Pipeline] . (step 1 of 3) Processing lowercase_speaker, total=   0.0s
[Pipeline]  (step 2 of 3) Processing connect_orphaned_lines_top10, total=   0.2s
[Pipeline]  (step 3 of 3) Processing remove_episode_s6e21, total=   0.0s


,season,episode,title,scene,speaker,line
0,1,1,Pilot,1,michael,All right Jim. Your quarterlies look very good...
1,1,1,Pilot,1,jim,"Oh, I told you. I couldn't close it. So..."
2,1,1,Pilot,1,michael,So you've come to the master for guidance? Is ...
3,1,1,Pilot,1,jim,"Actually, you called me in here, but yeah."
4,1,1,Pilot,1,michael,"All right. Well, let me show you how it's done."


Check our results:

In [140]:
# looking for all lowercase values and dwight=6752
script_df_cleaned['speaker'].value_counts().nlargest(10)

michael    10774
dwight      6752
jim         6223
pam         4973
andy        3698
kevin       1535
angela      1534
erin        1413
oscar       1336
ryan        1182
Name: speaker, dtype: int64

In [141]:
# looking for no "21" value in this list
script_df_cleaned.loc[script_df['season'] == 6, 'episode'].unique()

array([ 1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 19,
       20, 22, 23, 24, 25, 26], dtype=int64)